In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import locale
from datetime import datetime
from datetime import timedelta
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
# Load and prepare data
df = pd.read_csv("../../data/filtered_data/comments_posts_transcripts.csv")

# Remove posts with less than views
df = df[df["playCount"] > 50].copy()

df["createTimeISO"] = pd.to_datetime(df["createTimeISO"]).dt.tz_localize(None)

# Add week column
df["week"] = df["createTimeISO"].dt.to_period("W").dt.start_time

Engagement per skincare influencer

In [5]:
# Calculate engagement
df["engagement"] = df["diggCount"] + df["commentCount"] + df["shareCount"]

# Set up interactive widgets (standardized date range: 6 months)
max_date = df["createTimeISO"].max().date()
six_months_ago = max_date - timedelta(days=180)

start_date = widgets.DatePicker(description='Start Date', value=six_months_ago)
end_date = widgets.DatePicker(description='End Date', value=max_date)
min_posts_slider = widgets.IntSlider(value=3, min=1, max=20, step=1, description='Min Posts')

# Define interactive update function
def update_plot(start, end, min_posts):
    if start is None or end is None:
        print("Please select both start and end dates.")
        return

    start = pd.to_datetime(start)
    end = pd.to_datetime(end)

    if start > end:
        print("Start date must be before end date.")
        return

    # Filter data by date range
    mask = (df["createTimeISO"] >= start) & (df["createTimeISO"] <= end)
    filtered_df = df.loc[mask].copy()

    if filtered_df.empty:
        print("No data available for the selected date range.")
        return

    # Group by influencer
    grouped = (
        filtered_df.groupby("author_nickName")
        .agg({
            "engagement": "sum",
            "playCount": "sum",
            "author_fans": "first",
            "post_id": "count"
        })
        .rename(columns={"post_id": "num_posts"})
        .reset_index()
    )

    # Filter by minimum number of posts
    grouped = grouped[grouped["num_posts"] >= min_posts]

    if grouped.empty:
        print(f"No influencers with at least {min_posts} posts in this period.")
        return

    # Sort and select top 10
    top = grouped.sort_values(by="engagement", ascending=False).head(10)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(top["author_nickName"], top["engagement"])
    plt.xlabel("Total Engagement")
    plt.ylabel("Influencer")
    plt.title(f"Top 10 Influencers by Engagement\n({start.date()} to {end.date()}, Min Posts: {min_posts})")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Display interactive widgets
widgets.interact(update_plot, start=start_date, end=end_date, min_posts=min_posts_slider);


interactive(children=(DatePicker(value=datetime.date(2025, 1, 9), description='Start Date', step=1), DatePicke…

Spam phrases, viral memes phrases

In [6]:
# Ensure all necessary columns are included from the start
comments_df = df[df["comment"].notna()][["comment", "author_nickName", "createTimeISO"]].copy()

# Basic text cleaning: lowercase and strip whitespace
comments_df["comment_clean"] = comments_df["comment"].str.strip().str.lower()

# Remove exact duplicate comments
comments_df = comments_df.drop_duplicates(subset=["comment_clean"])

# Define known spam/meme phrases to filter out
spam_phrases = [
    "speed made you famous",
    "amy made speed",
    "speed speed",
    "famous speed made",
    "made speed famous",
    "speed made",
    "speed"
]
spam_pattern = "|".join(re.escape(p) for p in spam_phrases)

# Exclude comments that match any of the spam patterns
filtered_comments_df = comments_df[~comments_df["comment_clean"].str.contains(spam_pattern)]

# Optionally: filter out comments that contain 'speed' more than once
filtered_comments_df["speed_count"] = filtered_comments_df["comment_clean"].str.count(r"\bspeed\b")
filtered_comments_df = filtered_comments_df[filtered_comments_df["speed_count"] <= 1]

# Clean up: remove the helper column
filtered_comments_df = filtered_comments_df.drop(columns=["speed_count"])


/var/folders/5k/b8_r8jw502n4zztm42zd76m80000gn/T/ipykernel_39761/4199171522.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_comments_df["speed_count"] = filtered_comments_df["comment_clean"].str.count(r"\bspeed\b")


Common phrases in comments per influencer

In [7]:
# Define standard and custom stopwords
standard_stopwords = set(stopwords.words("german")) | set(stopwords.words("english"))


# Unigram-specific stopwords (standard + custom)
combined_stopwords = standard_stopwords

In [8]:
# Text input for influencer name
influencer_input = widgets.Text(
    description="Influencer:",
    placeholder="Enter name, e.g. amyflamy"
)

# Date pickers
today = df["createTimeISO"].max().date()
six_months_ago = today - timedelta(days=180)
start_ngram = widgets.DatePicker(description="Start Date", value=six_months_ago)
end_ngram = widgets.DatePicker(description="End Date", value=today)

# Function for dynamic bigram/trigram output
def show_ngrams_manual(influencer, start, end):
    clear_output(wait=True)

    if not influencer or start is None or end is None:
        print("Please enter an influencer name and valpostpost_id dates.")
        return

    # Filter comments
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)

    df_filtered = filtered_comments_df[
        (filtered_comments_df["author_nickName"] == influencer) &
        (filtered_comments_df["createTimeISO"] >= start) &
        (filtered_comments_df["createTimeISO"] <= end)
    ]

    if df_filtered.empty:
        print(f"No comments found for '{influencer}' in selected time range.")
        return

    # Tokenize and clean
    all_text = " ".join(df_filtered["comment"].dropna())
    tokens = [
        word for word in word_tokenize(all_text.lower())
        if re.match(r'^[a-z]{3,}$', word) and word not in combined_stopwords
    ]

    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))

    bigram_freq = Counter(bigrams).most_common(20)
    trigram_freq = Counter(trigrams).most_common(20)

    # Output
    print(f"Top 20 Bigrams for '{influencer}' ({start.date()} to {end.date()}):")
    for bg in bigram_freq:
        print("  ", " ".join(bg[0]), "-", bg[1])

    print(f"\nTop 20 Trigrams for '{influencer}':")
    for tg in trigram_freq:
        print("  ", " ".join(tg[0]), "-", tg[1])

# Create interactive output
out = widgets.interactive_output(
    show_ngrams_manual,
    {'influencer': influencer_input, 'start': start_ngram, 'end': end_ngram}
)

# Pack widgets in Layout
ui = widgets.VBox([influencer_input, start_ngram, end_ngram])

# Display UI and Output
display(ui, out)

Output()